# Drone Anomaly Detection Dataset

AAI-530 Group 2 Final Project

| Group Members |
|:---|
| Aliaksei Matsarski |
| Andrew Fennimore |

This notebook downloads the Drone Anomaly Detection Time Series Dataset from Kaggle, loads and inspects the data, runs some exploratory data analysis, and then splits it into train, validation, and test sets for modeling.

## Dataset Description

This dataset contains preprocessed time series data for a binary classification task. The goal is to determine whether a drone is healthy or faulty based on its motion data. The data has already been windowed with no missing values.

### Source

The data is derived from the "DronePropA: Motion Trajectories Dataset for Defective Drones" by Ismail, Elshaar, et al. The original dataset consists of 130 .mat files, each representing a single flight experiment. You can find the processed version on [Kaggle](https://www.kaggle.com/datasets/PirMustafa/drone-dataset).

### Preprocessing

The original .mat files were processed into a single .npz file. The following steps were applied (Information taken from the [Kaggle dataset page](https://www.kaggle.com/datasets/PirMustafa/drone-dataset)):

1. Feature Extraction: For each of the 130 flights, 12 specific time series features were extracted focusing on the drone's core motion dynamics.
2. Labeling: Each flight was labeled as healthy or faulty based on the file naming convention described in the source paper. Healthy is 0 and faulty is 1.
3. Windowing: The time series data from each flight was segmented into overlapping windows. Each window is 200 timesteps long with a 50% overlap between consecutive windows.
4. Aggregation: All windows from all flights were stacked into a single dataset.

### Structure

The data is contained in a single compressed NumPy archive file called processed_data.npz, which is around 637 MB. This file contains two arrays, X and y.

X is a 3 dimensional NumPy array containing the feature data:
- The first dimension is the total number of windows aggregated from all flights
- The second dimension is the number of timesteps in each window, which is 200
- The third dimension is the number of features recorded at each timestep, which is 12

y is a 1 dimensional NumPy array containing the corresponding labels for each window in X. A value of 0 means the window came from a healthy flight, and a value of 1 means it came from a faulty flight.

### Features

The 12 features in the third dimension of X are, in order:

1. Position X in meters
2. Position Y in meters
3. Position Z in meters
4. Roll in radians
5. Pitch in radians
6. Yaw in radians
7. Roll Rate in rad/s
8. Pitch Rate in rad/s
9. Yaw Rate in rad/s
10. Acceleration X in m/s squared
11. Acceleration Y in m/s squared
12. Acceleration Z in m/s squared

## 1. Prerequisites

Before running this notebook, make sure you have the following set up:

1. A Kaggle account (you can sign up at [kaggle.com](https://www.kaggle.com))
2. Install the required packages by running `pip install -r requirements.txt`
3. The first time you run this, `kagglehub` will ask you to log in through your browser. After that it remembers your credentials so you won't need to do it again.

In [ ]:
import kagglehub
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf

In [ ]:
# grab the dataset from kaggle, it will cache locally so it only downloads once
dataset_path = kagglehub.dataset_download("PirMustafa/drone-dataset")
print(f"Dataset downloaded to: {dataset_path}")

## 2. Load and Inspect the Data

Now that the dataset is downloaded, we can load the `.npz` file and take a look at what is inside.

In [ ]:
# load up the npz file
NPZ_FILE = os.path.join(dataset_path, "processed_data.npz")
data = np.load(NPZ_FILE)

# see what arrays are in here
print("Arrays in the .npz file:", list(data.keys()))

# pull out features and labels
X = data["X"]
y = data["y"]

print(f"\nFeatures X shape: {X.shape}")
print(f"  - {X.shape[0]} windows")
print(f"  - {X.shape[1]} timesteps per window")
print(f"  - {X.shape[2]} features per timestep")

print(f"\nLabels y shape: {y.shape}")
print(f"  - Unique labels: {np.unique(y)}")
print(f"  - Label distribution: 0 = healthy = {np.sum(y == 0)}, 1 = faulty = {np.sum(y == 1)}")

print(f"\nX dtype: {X.dtype}")
print(f"y dtype: {y.dtype}")

# check for missing values
nan_count = np.isnan(X).sum()
print(f"\nMissing values in X: {nan_count}")

# check for duplicate windows by flattening each window and comparing
X_2d = X.reshape(X.shape[0], -1)
n_duplicates = X_2d.shape[0] - np.unique(X_2d, axis = 0).shape[0]
print(f"Duplicate windows: {n_duplicates}")

# check labels for anything unexpected
print(f"\nUnique labels in y: {np.unique(y)}")
print(f"Missing values in y: {np.isnan(y.astype(float)).sum()}")

## 3. Display First Windows as DataFrames

Since `X` is a 3D array (windows x timesteps x features), each "row" is really a full window of 200 timesteps across 12 features. To get a better look at the actual values, we will convert the first two windows into pandas DataFrames with named columns and show the first and last few timesteps of each.

In [ ]:
# the 12 feature names in order
FEATURE_NAMES = [
    "Position_X", "Position_Y", "Position_Z",
    "Roll", "Pitch", "Yaw",
    "Roll_Rate", "Pitch_Rate", "Yaw_Rate",
    "Acceleration_X", "Acceleration_Y", "Acceleration_Z"
]

# show the first 2 windows as dataframes so we can eyeball the values
for i in range(2):
    window = X[i]  # 200 timesteps x 12 features
    df = pd.DataFrame(window, columns = FEATURE_NAMES)
    df.index.name = "Timestep"

    label_str = "Healthy" if y[i] == 0 else "Faulty"

    print(f"\n{"=" * 80}")
    print(f"Window {i} | Label: {y[i]} - {label_str}")
    print(f"{"=" * 80}")

    display(df.head(5))
    print(f"  ... {len(df) - 10} rows omitted ...")
    display(df.tail(5))

## 4. Basic Summary Statistics

Here we run some quick sanity checks on the feature ranges and look at how the labels are distributed across the dataset.

In [ ]:
# flatten X down to 2D so we can run describe across all timesteps
X_flat = X.reshape(-1, X.shape[2])
stats_df = pd.DataFrame(X_flat, columns = FEATURE_NAMES).describe()

# format the table so it doesnt show scientific notation
print("Overall feature statistics across all windows and timesteps:")
display(stats_df.style.format("{:.4f}"))

# check the label balance
total = len(y)
healthy = np.sum(y == 0)
faulty = np.sum(y == 1)

print(f"\nLabel balance:")
print(f"  Healthy: {healthy} - {100 * healthy / total:.1f}%")
print(f"  Faulty:  {faulty} - {100 * faulty / total:.1f}%")

## 5. Exploratory Data Analysis

In this section we visualize the class balance, look at how the 12 features are distributed across healthy and faulty windows, and plot a sample window from each class to see what the raw time series data looks like.

In [ ]:
# bar chart for class balance
labels, counts = np.unique(y, return_counts = True)
label_names = ["Healthy", "Faulty"]
colors = ["#3498db", "#e74c3c"]

fig, ax = plt.subplots(figsize = (6, 5))
bars = ax.bar(label_names, counts, color = colors, edgecolor = "black", width = 0.5)

for bar, count in zip(bars, counts):
    pct = 100 * count / len(y)
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() / 2,
        f"{count}\n{pct:.1f}%",
        ha = "center", va = "center", fontweight = "bold", fontsize = 11, color = "white"
    )

ax.set_ylabel("Number of Windows")
ax.set_title("Class Balance: Healthy vs Faulty Windows")
plt.tight_layout()
plt.show()

In [ ]:
# feature distributions using the per-window mean, split by class
X_mean = X.mean(axis = 1)  # average each window down to 12 values

fig, axes = plt.subplots(4, 3, figsize = (15, 12))
axes = axes.flatten()

for i, (ax, name) in enumerate(zip(axes, FEATURE_NAMES)):
    ax.hist(X_mean[y == 0, i], bins = 50, alpha = 0.6, color = "#3498db", label = "Healthy", density = True)
    ax.hist(X_mean[y == 1, i], bins = 50, alpha = 0.6, color = "#e74c3c", label = "Faulty", density = True)
    ax.set_title(name)
    ax.set_ylabel("Density")

    if i == 0:
        ax.legend()

fig.suptitle("Feature Distributions by Class", fontsize = 14, y = 1.01)
plt.tight_layout()
plt.show()

## 6. Train / Validation / Test Split

We split the data into 70% training, 15% validation, and 15% test sets using stratified sampling so the class balance stays consistent across all three sets. Since the dataset comes pre windowed and we do not have flight IDs, we cannot do a per flight split, so a stratified random split is the best we can do here.

In [ ]:
# first split: 70% train, 30% temp
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size = 0.30, random_state = 1, stratify = y
)

# second split: chop the 30% in half for 15% val and 15% test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size = 0.50, random_state = 1, stratify = y_temp
)

# print out shapes and class balance for each split
for name, X_split, y_split in [("Train", X_train, y_train),
                                ("Validation", X_val, y_val),
                                ("Test", X_test, y_test)]:
    n = len(y_split)
    h = np.sum(y_split == 0)
    f = np.sum(y_split == 1)

    print(f"{name:>10s}: X = {X_split.shape}, y = {y_split.shape}  |  "
          f"Healthy = {h} - {100 * h / n:.1f}%  Faulty = {f} - {100 * f / n:.1f}%")

print(f"\n  Total: {len(y_train)} + {len(y_val)} + {len(y_test)} = {len(y_train) + len(y_val) + len(y_test)}")

### Export Splits

We save each split as a `.npz` file in the `data/` directory so they are easy to load later during model training.

In [ ]:
# save each split to the data folder as npz files
DATA_DIR = os.path.join(os.getcwd(), "data")
os.makedirs(DATA_DIR, exist_ok = True)

np.savez(os.path.join(DATA_DIR, "train.npz"), X = X_train, y = y_train)
np.savez(os.path.join(DATA_DIR, "validation.npz"), X = X_val, y = y_val)
np.savez(os.path.join(DATA_DIR, "test.npz"), X = X_test, y = y_test)

print("Saved to data/:")

for name in ["train.npz", "validation.npz", "test.npz"]:
    d = np.load(os.path.join(DATA_DIR, name))
    print(f"  {name}: X = {d['X'].shape}, y = {d['y'].shape}")

## 7. LSTM Model

Now we can build and train an LSTM to classify each window as healthy or faulty. The model takes in the full sequence of 200 timesteps across all 12 features and learns to pick up on patterns that separate the two classes. We wrap everything in a class so its easy to train and make predictions with.

In [ ]:
class LSTM:
    def __init__(self, num_units, n_classes, n_input,
                 time_steps, learning_rate = 0.001):
        self.steps = time_steps
        self.n = n_input

        # build the lstm model
        self.model = tf.keras.Sequential([
            tf.keras.layers.LSTM(num_units, input_shape = (time_steps, n_input)),
            tf.keras.layers.Dense(n_classes, activation = "softmax")
        ])

        # compile with adam optimizer and cross entropy loss
        self.model.compile(
            optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
            loss = "sparse_categorical_crossentropy",
            metrics = ["accuracy"]
        )

    def train(self, X, Y, epochs = 100, batch_size = 128):
        # reshape input to match what the lstm expects
        X = X.reshape((len(X), self.steps, self.n))

        # train the model
        self.history = self.model.fit(X, Y, epochs = epochs, batch_size = batch_size)

    def predict(self, X):
        # predicting the output
        test_data = X.reshape((-1, self.steps, self.n))
        out = self.model.predict(test_data)
        return out

In [ ]:
# load the split data from the saved npz files
DATA_DIR = os.path.join(os.getcwd(), "data")
train_data = np.load(os.path.join(DATA_DIR, "train.npz"))
val_data = np.load(os.path.join(DATA_DIR, "validation.npz"))
test_data = np.load(os.path.join(DATA_DIR, "test.npz"))

X_train, y_train = train_data["X"], train_data["y"]
X_val, y_val = val_data["X"], val_data["y"]
X_test, y_test = test_data["X"], test_data["y"]

# create the lstm model
# 128 hidden units, 2 classes, 12 features, 200 timesteps
model = LSTM(
    num_units = 128,
    n_classes = 2,
    n_input = 12,
    time_steps = 200,
    learning_rate = 0.001
)

# train on the training data
model.train(X_train, y_train, epochs = 50, batch_size = 128)

In [ ]:
# run predictions on the test set
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis = 1)

# check how we did
test_accuracy = np.mean(predicted_labels == y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

print(f"\nConfusion Matrix:")
print(confusion_matrix(y_test, predicted_labels))

print(f"\nClassification Report:")
print(classification_report(y_test, predicted_labels, target_names = ["Healthy", "Faulty"]))

## Next Steps

At this point the dataset has been downloaded, inspected, explored, and split into train, validation, and test sets. From here we can move on to:

- Model selection and training (LSTM, GRU, 1D CNN, etc.)
- Hyperparameter tuning
- Evaluation and comparison of results